# Panel Regression - Firm Characteristics

### Random Effects Panel Regression

In [15]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects

Data

In [16]:
df = pd.read_csv("Dataframes/analysts_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,Recommendation - Mean (1-5).1,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,3.625620,7.0,3.00000,0.00000,3.000,2.44949
1,AVY.N,2013-04-01,Materials,2.482,2.482,2.046169,7.0,2.87500,-0.12500,3.000,3.92272
2,AVY.N,2013-07-01,Materials,1.068,1.068,1.551601,8.0,2.87500,0.00000,2.875,4.30946
3,AVY.N,2013-10-01,Materials,8.095,8.095,2.775994,7.0,2.62500,-0.25000,2.875,3.45230
4,AVY.N,2014-01-01,Materials,1.471,1.471,3.204412,8.0,2.33333,-0.29167,2.625,3.42555
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,2.836295,8.0,2.30000,0.00000,2.300,50.48823
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,3.269867,8.0,2.30000,0.00000,2.300,41.33833
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,6.630736,9.0,2.00000,-0.30000,2.300,37.49206
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,3.625249,9.0,2.20000,0.20000,2.000,57.57373


Standardising and log transformation

In [17]:
rescale = df.copy()
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Earnings Per Share – Coefficient of Variation"] = StandardScaler().fit_transform(np.array(rescale["Earnings Per Share – Coefficient of Variation"]).reshape(-1,1))
rescale["Number of Analysts"] = StandardScaler().fit_transform(np.array(rescale["Number of Analysts"]).reshape(-1,1))
rescale["Recommendation - Mean (1-5)"] = StandardScaler().fit_transform(np.array(rescale["Recommendation - Mean (1-5)"]).reshape(-1,1))
rescale["Recommendation change"] = StandardScaler().fit_transform(np.array(rescale["Recommendation change"]).reshape(-1,1))
rescale["Recommendation - Mean (1-5).1"] = StandardScaler().fit_transform(np.array(rescale["Recommendation - Mean (1-5).1"]).reshape(-1,1))
rescale["Price Target - Standard Deviation"] = StandardScaler().fit_transform(np.array(rescale["Price Target - Standard Deviation"]).reshape(-1,1))

#log transform DV
rescale["Earnings Per Share - Actual Surprise AbsVals"] = np.log(rescale["Earnings Per Share - Actual Surprise AbsVals"] + 1)

df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,Recommendation - Mean (1-5).1,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,2.499631,-0.063585,-1.456340,1.793515,0.002057,1.789773,-0.371337
1,AVY.N,2013-04-01,Materials,2.482,1.247607,-0.111877,-1.456340,1.461311,-0.990304,1.789773,-0.313057
2,AVY.N,2013-07-01,Materials,1.068,0.726582,-0.126998,-1.319027,1.461311,0.002057,1.457006,-0.297757
3,AVY.N,2013-10-01,Materials,8.095,2.207725,-0.089562,-1.456340,0.796901,-1.982665,1.457006,-0.331666
4,AVY.N,2014-01-01,Materials,1.471,0.904623,-0.076463,-1.319027,0.021748,-2.313479,0.791472,-0.332724
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,2.901092,-0.087719,-1.319027,-0.066832,0.002057,-0.073723,1.529043
19197,POOL.OQ,2022-01-01,Retailing,40.267,3.720063,-0.074462,-1.319027,-0.066832,0.002057,-0.073723,1.167079
19198,POOL.OQ,2022-04-01,Retailing,34.342,3.565072,0.028297,-1.181715,-0.864123,-2.379610,-0.073723,1.014923
19199,POOL.OQ,2022-07-01,Retailing,1.503,0.917490,-0.063596,-1.181715,-0.332595,1.589834,-0.872364,1.809340


In [18]:
#df_clean = df_clean.drop(["Recommendation - Mean (1-5)","Recommendation - Mean (1-5).1"], axis=1)
#df_clean

In [19]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [20]:
#reformatting indices of dataframe for panel regression
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
#df_clean

FE Panel Regression with **Absolute** Surprise Values

In [21]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0115
Estimator:,PanelOLS,R-squared (Between):,-0.0221
No. Observations:,18208,R-squared (Within):,0.0121
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-0.0014
Time:,20:20:04,Log-likelihood,-2.378e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,41.001
Entities:,502,P-value,0.0000
Avg Obs:,36.271,Distribution:,"F(5,17663)"
Min Obs:,0.0000,,
Max Obs:,40.000,F-statistic (robust):,1.946e+04


By Industry: example tech industry

In [22]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "Earnings Per Share – Coefficient of Variation":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74461/2540156368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74461/2540156368.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])
/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs

Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0939
Estimator:,PanelOLS,R-squared (Between):,-0.2181
No. Observations:,603,R-squared (Within):,0.0971
Date:,"Tue, Mar 07 2023",R-squared (Overall):,0.0238
Time:,20:20:04,Log-likelihood,-661.49
Cov. Estimator:,Unadjusted,,
,,F-statistic:,11.188
Entities:,19,P-value,0.0000
Avg Obs:,31.737,Distribution:,"F(5,540)"
Min Obs:,1.0000,,
Max Obs:,40.000,F-statistic (robust):,531.55


# COVID Tests

In [23]:
df_clean = df_panel.copy()
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,Recommendation - Mean (1-5).1,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,2.499631,-0.063585,-1.456340,1.793515,0.002057,1.789773,-0.371337
1,AVY.N,2013-04-01,Materials,2.482,1.247607,-0.111877,-1.456340,1.461311,-0.990304,1.789773,-0.313057
2,AVY.N,2013-07-01,Materials,1.068,0.726582,-0.126998,-1.319027,1.461311,0.002057,1.457006,-0.297757
3,AVY.N,2013-10-01,Materials,8.095,2.207725,-0.089562,-1.456340,0.796901,-1.982665,1.457006,-0.331666
4,AVY.N,2014-01-01,Materials,1.471,0.904623,-0.076463,-1.319027,0.021748,-2.313479,0.791472,-0.332724
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,2.901092,-0.087719,-1.319027,-0.066832,0.002057,-0.073723,1.529043
19197,POOL.OQ,2022-01-01,Retailing,40.267,3.720063,-0.074462,-1.319027,-0.066832,0.002057,-0.073723,1.167079
19198,POOL.OQ,2022-04-01,Retailing,34.342,3.565072,0.028297,-1.181715,-0.864123,-2.379610,-0.073723,1.014923
19199,POOL.OQ,2022-07-01,Retailing,1.503,0.917490,-0.063596,-1.181715,-0.332595,1.589834,-0.872364,1.809340


In [24]:
covid_start = pd.to_datetime("2020-01-01")

df_covid = df_clean[df_clean["Date"] >= covid_start]

df_nocovid = df_clean[df_clean["Date"] < covid_start]

In [25]:
df_covid['Instrument'] = df_covid['Instrument'].astype('category')
df_covid['Date'] = pd.to_datetime(df_covid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_covid.set_index(['Instrument', 'Date'], inplace=True)

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74461/2704597447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['Instrument'] = df_covid['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74461/2704597447.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['Date'] = pd.to_datetime(df_covid['Date'])


In [26]:
df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_nocovid.set_index(['Instrument', 'Date'], inplace=True)

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74461/1939219620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74461/1939219620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])


## COVID

In [27]:
y = df_covid.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_covid.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0054
Estimator:,PanelOLS,R-squared (Between):,-0.0031
No. Observations:,5450,R-squared (Within):,0.0044
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-0.0042
Time:,20:20:05,Log-likelihood,-6963.0
Cov. Estimator:,Unadjusted,,
,,F-statistic:,5.3814
Entities:,501,P-value,0.0001
Avg Obs:,10.878,Distribution:,"F(5,4934)"
Min Obs:,0.0000,,
Max Obs:,12.000,F-statistic (robust):,7478.2


In [28]:
y = df_nocovid.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_nocovid.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0091
Estimator:,PanelOLS,R-squared (Between):,-0.0116
No. Observations:,12758,R-squared (Within):,0.0091
Date:,"Tue, Mar 07 2023",R-squared (Overall):,0.0009
Time:,20:20:05,Log-likelihood,-1.598e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,22.602
Entities:,498,P-value,0.0000
Avg Obs:,25.618,Distribution:,"F(5,12266)"
Min Obs:,0.0000,,
Max Obs:,28.000,F-statistic (robust):,1.129e+04


### Based on these results we can conclude that Analyst factors are a bad indicator for prediction